In [ ]:
#                                MASTER THESIS 

# Topic: MACHINE LEARNING APPROACHES TO THE INVERSE PROBLEM OF IDENTIFYING CRACKS FROM ELECTRICAL SIGNALS IN STRUCTURAL HEALTH MONITORING"

# Programmed by: Vundurthy, Padmanabha Pavan Chandra.

# Course: M.Sc. Computational Materials Science.

# Technische Universitaet Bergakademie Freiberg, Freiberg

# Date: July 2nd, 2021

## This program generates virtual training data for hexagonal layout with 13 sensors for 5 sample points within the training range of parameters. 

#Import numpy and pandas modules
import numpy as np
import pandas as pd

###Generate virtual training data 
##Assign variables for crack coordinates, angle, stress intensity factors and T stress
x0_all,y0_all,b0_all,K_1_all,K_2_all,T_all=([] for i in range(6))

##Number of points considered in each parameter
num_points=5 

#Parameters in sequence:SIF-I, SIF-II, T-stress, angle & Coordinates of crack(y&x)
##Ranges for data parameters: 
#K_I=0 to 1200; K_II=-110 to 130; T-stress=-60 to 22; angle=0 to 2*pi; x_coordinate=-17 to 17 and y_coordinate= -17 to 17

K_I_range =np.linspace(0,1200,num_points)
K_II_range=np.linspace(-110,130,num_points)
T_range =np.linspace(-60,22,num_points)
Beta_range =np.linspace(0,2*np.pi,num_points)
Y_coord_range =np.linspace(-17,17,num_points)
X_coord_range =np.linspace(-17,17,num_points)

#Looping over to create data
for a in K_I_range:
    for b in K_II_range:
        for c in T_range:
            for angle in Beta_range:    
                for j in Y_coord_range:    
                    for i in X_coord_range:
                        x0_all.append(i)        
                        y0_all.append(j)
                        b0_all.append(angle)
                        K_1_all.append(a)
                        K_2_all.append(b)
                        T_all.append(c)

##Obtaing strain field for varying x_0 and y_0 generated above
count_2 = 0
for count_1 in range(0,len(x0_all)):
    x_0=x0_all[count_1]
    y_0=y0_all[count_1]
    beta = b0_all[count_1]
    SIF_1=K_1_all[count_1]
    SIF_2=K_2_all[count_1]
    T=T_all[count_1]
    
    ## Number of sensors in hexagonal layout 
    num_sensors=13 
    #Assigning variables for sensor arrangement coordinate system (l.c.s)
    x_dash = np.zeros(num_sensors)
    y_dash = np.zeros(num_sensors)
    #Assigning variables for crack coordinate system (g.c.s)
    x = np.zeros(num_sensors)
    y = np.zeros(num_sensors)
    phi = np.zeros(num_sensors)
    x_count = 0
    y_count = 0
    
    #Side of a hexagon : a_dist
    a_dist = 20 

    count = 0
    x_dash[count]= 0
    y_dash[count]= 0
    for k in np.linspace(0, 5*np.pi/3, 6):
        count = count + 1
        x_dash[count]= 0 + a_dist*np.cos(k)
        y_dash[count]= 0 + a_dist*np.sin(k)
        
    for l in np.linspace(np.pi/6, 11*np.pi/6, 6):
        count = count + 1
        x_dash[count]= 0 + (a_dist/np.sqrt(3))*np.cos(l)
        y_dash[count]= 0 + (a_dist/np.sqrt(3))*np.sin(l)

    #using the relation between g.c.s and l.c.s, g.c.s values are obtained from l.c.s values and crack position
    for i in range(0,num_sensors):
        x[i] = x_dash[i]*np.cos(beta)-y_dash[i]*np.sin(beta)+x_0
        y[i] = y_dash[i]*np.sin(beta)+y_dash[i]*np.cos(beta)+y_0 

    #coordinates of the measuring points (r,phi)
    r = np.sqrt(x**2+y**2)
    phi = np.arctan2(y,x)

    #computations from equations mentioned in [Sensor concept based on piezoelectric PVDF films for SHM of fatigue crack growth]
    nu = 0.33
    E = 72

    #with warnings.catch_warnings():
        #warnings.filterwarnings('error')
        
    #computing stress values from the equations        
    np.seterr(divide='raise')
    try:
        stress_11 = ((SIF_1/(np.sqrt(2*np.pi*r))) * (np.cos(phi/2)) * (1-np.sin(phi/2)*np.sin((3*phi)/2))) - ((SIF_2/(np.sqrt(2*np.pi*r))) * np.sin(phi/2) * (2+np.cos(phi/2)*np.cos(3*phi/2))) + T
        stress_22 = ((SIF_1/(np.sqrt(2*np.pi*r))) * (np.cos(phi/2)) * (1+np.sin(phi/2)*np.sin((3*phi)/2))) + ((SIF_2/(np.sqrt(2*np.pi*r))) * np.sin(phi/2) * (np.cos(phi/2) * np.cos(3*phi/2)))
        stress_12 = ((SIF_1/(np.sqrt(2*np.pi*r))) * (np.sin(phi/2)) * (np.cos(phi/2) * np.cos((3*phi)/2))) + ((SIF_2/(np.sqrt(2*np.pi*r))) * np.cos(phi/2) * (1-(np.sin(phi/2)*np.sin(3*phi/2))))

    except FloatingPointError:
        continue
 
    #computing corresponding strains
    eps_specimen_11 = (stress_11-(nu*stress_22))/E
    eps_specimen_22 = (stress_22-(nu*stress_11))/E
    eps_specimen_12 = (1+nu)*stress_12/E

    #stacking strain values as input data (x_data) and corresponding [crack locations, cos(beta), sin(beta), SIF_1, SIF_2, T] as output data (y_data).
    stack_arr = np.column_stack((eps_specimen_11,eps_specimen_22,eps_specimen_12))
    xgroup = stack_arr.flatten()
    ygroup = np.column_stack((x_0,y_0,np.cos(beta),np.sin(beta),SIF_1,SIF_2,T))


    if (count_2 == 0):
        final_xgroup = xgroup
        final_ygroup = ygroup
        count_2 = 1
    else:
        final_xgroup = np.vstack((final_xgroup, xgroup))
        final_ygroup = np.vstack((final_ygroup, ygroup))
        
x_data = final_xgroup
y_data = final_ygroup

print (np.shape(x_data),"shape of x_data") #16 sensors and 5 outputs for each sensor, so 80
print(np.shape(y_data),"shape of y_data") # crack location(x and y) and angle(so 3 columns)

## Saving the generated data into two files separately as input data and output data for a hexagonal layout: filename= "x_(layout)_data_(number of points)"
df_x = pd.DataFrame(data=x_data)
df_y = pd.DataFrame(data=y_data)
df_x.to_csv('x_hex_data_5.csv')
df_y.to_csv('y_hex_data_5.csv')

###TRAINING DATA IS GENERATED FOR HEXAGONAL LAYOUT OF DESIRED DIMENSIONS ###